In [35]:
!pip install google-cloud-aiplatform
!pip install langchain
!pip install chromadb
!pip install langchain_experimental
!pip install transformers

  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/1a/d1/3bba59606141ae808017f6fde91453882f931957f125009417b87a281067/transformers-4.34.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 2.7 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for regex!=2019.12.17 from https://files.pythonhosted.org/packages/2c/6b/4828fdbbabcb51986ddc1e7c618cf9dc8606f75c2f0cc381d3729cf31348/regex-2023.10.3-cp310-cp310-macosx_10_9_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.4 MB/s eta 0:00:00
  Obtaining dependency information for safetensors>=0.3.1 from https://files.pythonhosted.org/packages/79/dc/a72eb060f4f598314c5a96676d3790b51fff78e448253c7319a9441228a7/safetensors-0.4.0-cp310-cp310-macosx_10_7_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 6.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 6.6 MB/s eta 

In [20]:
import os
from vertexai.preview.language_models import TextGenerationModel
from langchain import PromptTemplate

In [21]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/foochuanyue/Desktop/aiap-14/all-assignments/assignment7/aiap-14-ds-llm-topic-01-sa.json'

model = TextGenerationModel.from_pretrained("text-bison@001")
response = model.predict("Is the earth round?")

print(f"Response from Model: {response.text}")

Response from Model: The earth is round.

The earth is a sphere.


In [1]:
import sys
from langchain.llms import VertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore import InMemoryDocstore
from langchain.document_loaders.csv_loader import CSVLoader

In [12]:
sys.path.append("..")
from src.generative_agents.generative_agent import StemssGenerativeAgent
from src.generative_agents.memory import StemssGenerativeAgentMemory
from src.retrievers.time_weighted_retriever import ModTimeWeightedVectorStoreRetriever
from src.vectorstores.chroma import EnhancedChroma

In [14]:
def create_new_memory_retriever(decay_rate: float = 0.5, k: int = 5):
    """Create a new vector store retriever unique to the agent."""
    # Define your embedding model
    embeddings_model = VertexAIEmbeddings()
    vs = EnhancedChroma(embedding_function=embeddings_model)
    return ModTimeWeightedVectorStoreRetriever(
        vectorstore=vs, other_score_keys=["importance"], decay_rate=decay_rate, k=k
    )

In [18]:
mem_file = "../memory/memory.csv"
loader = CSVLoader(
        mem_file, metadata_columns=["last_accessed_at", "created_at", "importance"]
    )
docs = loader.load()

In [22]:
llm = VertexAI()
memory_retriever = create_new_memory_retriever(decay_rate=0.95, k=5)
memory_retriever.add_documents(docs)

['29c4917e-699e-11ee-91c3-acde48001122',
 '29c492aa-699e-11ee-91c3-acde48001122',
 '29c492dc-699e-11ee-91c3-acde48001122',
 '29c49304-699e-11ee-91c3-acde48001122',
 '29c49322-699e-11ee-91c3-acde48001122',
 '29c49340-699e-11ee-91c3-acde48001122',
 '29c49368-699e-11ee-91c3-acde48001122',
 '29c49386-699e-11ee-91c3-acde48001122',
 '29c493a4-699e-11ee-91c3-acde48001122',
 '29c493c2-699e-11ee-91c3-acde48001122',
 '29c493e0-699e-11ee-91c3-acde48001122']

In [23]:
tommies_memory = StemssGenerativeAgentMemory(
        llm=llm,
        memory_retriever=memory_retriever,
        reflection_threshold=8,  # we will give this a relatively low number to show how reflection works
        verbose=True,
    )

In [27]:
tommie = StemssGenerativeAgent(
    name="Tommie",
    age=25,
    traits="anxious, likes design, talkative",  # You can add more persistent traits here
    status="looking for a job",  # When connected to a virtual world, we can have the characters update their status
    llm=VertexAI(),
    memory=tommies_memory,
    background="You are an occult enthusiast who makes vlogs of you going to various allegedly haunted places. After graduating with a software engineering degree 10 years ago, you decided that ghosts were more interesting to you and now you make a decent living through your videos. In a sense, you are a small celebrity."
)

In [28]:
print(tommie.get_summary())

Number of requested results 100 is greater than number of elements in index 11, updating n_results = 11


Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is an adventurous and curious person, who enjoys exploring the unknown and sharing their experiences with others. They are also creative and have a passion for making videos. Additionally, Tommie is resilient and determined, as they have been able to turn their passion into a successful career.


In [29]:
# We can add memories directly to the memory object
tommie_observations = [
    "Tommie remembers his dog, Bruno, from when he was a kid",
    "Tommie feels tired from driving so far",
    "Tommie sees the new home",
    "The new neighbors have a cat",
    "The road is noisy at night",
    "Tommie is hungry",
    "Tommie tries to get some rest.",
]
for observation in tommie_observations:
    tommie.memory.add_memory(observation)



> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is not important at all (e.g., brushing teeth, making bed) and 10 is extremely important (e.g., a break up, college acceptance), rate the importance of the following piece of memory. You must respond with a single integer.
Memory: Tommie remembers his dog, Bruno, from when he was a kid
Rating: 


Importance score: 8



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is not important at all (e.g., brushing teeth, making bed) and 10 is extremely important (e.g., a break up, college acceptance), rate the importance of the following piece of memory. You must respond with a single integer.
Memory: Tommie feels tired from driving so far
Rating: 


Importance score: 2



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is not important at all (e.g., brushing teeth, making bed) and 10 is extremely important (e.g., a break up, college acceptance), rate the importance of the following piece of memory. You must respond with a single integer.
Memory: Tommie sees the new home
Rating: 


...


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is not important at all (e.g., brushing teeth, making bed) and 10 is extremely important (e.g., a break up, college acceptance), rate the importance of the following piece of memory. You must respond with a single integer.
Memory: Tommie is hungry
Rating: 


Importance score: 5



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is not important at all (e.g., brushing teeth, making bed) and 10 is extremely important (e.g., a break up, college acceptance), rate the importance of the following piece of memory. You must respond with a single integer.
Memory: Tommie tries to get some rest.
Rating: 


Importance score: 2



> Finished chain.


In [30]:
print(tommie.get_summary(force_refresh=True))

Number of requested results 100 is greater than number of elements in index 18, updating n_results = 18


Name: Tommie (age: 25)
Innate traits: anxious, likes design, talkative
Tommie is an adventurous and curious person, who enjoys exploring the unknown and sharing their experiences with others. They are also creative and have a passion for making videos. Additionally, Tommie is resilient and determined, as they have been able to turn their passion into a successful career.


In [36]:
print(tommie.generate_dialogue_response("Do you like driving?"))

Number of requested results 100 is greater than number of elements in index 18, updating n_results = 18
Number of requested results 100 is greater than number of elements in index 18, updating n_results = 18
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.




> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is not important at all (e.g., brushing teeth, making bed) and 10 is extremely important (e.g., a break up, college acceptance), rate the importance of the following piece of memory. You must respond with a single integer.
Memory: Tommie observed Do you like driving? and said "I don't really like driving, but I don't mind it. I prefer to be a passenger so I can relax and enjoy the ride."
Rating: 


Importance score: 2



> Finished chain.
(True, 'Tommie said "I don\'t really like driving, but I don\'t mind it. I prefer to be a passenger so I can relax and enjoy the ride."')


In [37]:
print(tommie.generate_dialogue_response("Who is Bruno?"))

Number of requested results 100 is greater than number of elements in index 19, updating n_results = 19
Number of requested results 100 is greater than number of elements in index 19, updating n_results = 19




> Entering new LLMChain chain...
Prompt after formatting:
On the scale of 1 to 10, where 1 is not important at all (e.g., brushing teeth, making bed) and 10 is extremely important (e.g., a break up, college acceptance), rate the importance of the following piece of memory. You must respond with a single integer.
Memory: Tommie observed Who is Bruno? and said Bruno is my dog. I had him when I was a kid. He was a good dog. I miss him.
Rating: 


Importance score: 8



> Finished chain.
(True, 'Tommie said Bruno is my dog. I had him when I was a kid. He was a good dog. I miss him.')
